In [2]:
import pandas as pd
import os

############################################# Utilised datasets
#These are all 'event' factors, they are the same for every accident.
# Accident_location.csv:
# Accident.csv:
# Atmospheric_cond.csv:
# Node.csv:
# Road_surface_cond.csv:


############################################# Mappings
# These are simply value mappings for the categorical data, just for reference.

node_type_from_id = {
    "I": "Intersection",
    "N": "Non-Intersection",
    "O": "Off Road",
    "U": "Unknown"
}

light_condition_from_id = {
    1: "Day",
    2: "Dusk/dawn",
    3: "Dark street lights on",
    4: "Dark street lights off",
    5: "Dark no street lights",
    6: "Dark street lights unknown",
    9: "Unknown"
}

road_geometry_from_id = {
    1: "Cross intersection",
    2: "T Intersection",
    3: "Y Intersection",
    4: "Multiple intersections",
    5: "Not at intersection",
    6: "Dead end",
    7: "Road closure",
    8: "Private property",
    9: "Unknown"
}

atmospheric_cond_from_id = {
    1: "Clear",
    2: "Raining",
    3: "Snowing",
    4: "Fog",
    5: "Smoke",
    6: "Dust",
    7: "Strong winds",
    9: "Not known"
}

surface_cond_from_id = {
    1: "Dry",
    2: "Wet",
    3: "Muddy",
    4: "Snowy",
    5: "Icy",
    9: "Unknown"
}

day_from_id = {
    1: "Sunday",
    2: "Monday",
    3: "Tuesday",
    4: "Wednesday",
    5: "Thursday",
    6: "Friday",
    7: "Saturday"
}


DATA_DIR = "data"



def event_preprocess():

    #181055 rows/accidents, keeping only important columns.
    accident_df = pd.read_csv("/Users/armaansidhu/Documents/Uni/year-2/EODP/code/proj_2/datasets/accident.csv")
    COLS_TO_KEEP = ["ACCIDENT_NO", "ACCIDENT_TIME", "DAY_OF_WEEK", "LIGHT_CONDITION", "ROAD_GEOMETRY", "SPEED_ZONE", "SEVERITY"]
    accident_df = accident_df[COLS_TO_KEEP]


    #181055 rows/accidents, again keeping only important columns.
    accident_location_df = pd.read_csv("/Users/armaansidhu/Documents/Uni/year-2/EODP/code/proj_2/datasets/accident_location.csv")
    COLS_TO_KEEP = ["ACCIDENT_NO", "ROAD_TYPE"]
    accident_location_df = accident_location_df[COLS_TO_KEEP]


    #183405 rows/accidents before pre-processing. This is because the same accident can have multiple atmospheric conditions. Fixed now to 181055 rows, with ATMOSPH_COND now containing a list of atmospheric conditions for each accident.
    atmospheric_cond_df = pd.read_csv("/Users/armaansidhu/Documents/Uni/year-2/EODP/code/proj_2/datasets/atmospheric_cond.csv")
    filtered_atmospheric_cond_df = atmospheric_cond_df.groupby('ACCIDENT_NO')['ATMOSPH_COND'].agg(list).reset_index()



    #182062 rows/accidents for the same reason as the previous, fixed to be 181055 rows.
    road_surface_cond_df = pd.read_csv("/Users/armaansidhu/Documents/Uni/year-2/EODP/code/proj_2/datasets/road_surface_cond.csv")
    filtered_road_surface_cond_df = road_surface_cond_df.groupby('ACCIDENT_NO')['SURFACE_COND'].agg(list).reset_index()


    #183830 rows/accidents for the same reason as the previous ones. Its a little weird though, they are saying
    #the same accident can have multiple locations which seems kind of stupid. 
    #NOTE: for now, I literally just take the first instance of location for each accident.
    node_df = pd.read_csv("/Users/armaansidhu/Documents/Uni/year-2/EODP/code/proj_2/datasets/node.csv")
    columns_to_keep = ['ACCIDENT_NO', 'NODE_TYPE', 'DEG_URBAN_NAME', 'POSTCODE_CRASH']
    filtered_node_df = node_df[columns_to_keep]


    #Now we get 180970 rows which is less than the 183405 accidents. This means there are 85 accidents that are in all of the other dataframes but not in the node dataframe. I just remove these from the total dataframe for now, giving us a total of 180970 rows.
    filtered_node_df = filtered_node_df.drop_duplicates(subset='ACCIDENT_NO')


    #We copy from the node df that doesn't contain those 85 accidents, so all operations will leave out the bad 85 accidents.
    event_df = filtered_node_df.copy()
    event_df = event_df.merge(accident_df, on='ACCIDENT_NO', how='inner')
    event_df = event_df.merge(accident_location_df, on='ACCIDENT_NO', how='inner')
    event_df = event_df.merge(filtered_atmospheric_cond_df, on='ACCIDENT_NO', how='inner')
    event_df = event_df.merge(filtered_road_surface_cond_df, on='ACCIDENT_NO', how='inner')

    #Move the severity column to the end just for clarity:
    event_df = event_df[[col for col in event_df.columns if col != 'SEVERITY'] + ['SEVERITY']]

    #We have 180970 rows and 13 columns.
    event_df.to_csv("event_df.csv", index=False)

    return event_df


if __name__ == "__main__":  
    event_df = event_preprocess()
    print(event_df)

         ACCIDENT_NO NODE_TYPE  DEG_URBAN_NAME  POSTCODE_CRASH ACCIDENT_TIME  \
0       T20120000009         N  RURAL_VICTORIA            3981      02:25:00   
1       T20120000012         N      MELB_URBAN            3170      02:00:00   
2       T20120000013         I      MELB_URBAN            3169      03:35:00   
3       T20120000018         I  RURAL_VICTORIA            3505      05:15:00   
4       T20120000021         N      MELB_URBAN            3942      07:30:00   
...              ...       ...             ...             ...           ...   
178603  T20250000840         N      MELB_URBAN            3054      18:50:00   
178604  T20250001075         N      MELB_URBAN            3022      16:30:00   
178605  T20250001671         I      MELB_URBAN            3073      09:11:00   
178606  T20250001949         N    SMALL_CITIES            3212      07:28:00   
178607  T20250002210         I     SMALL_TOWNS            3809      09:20:00   

        DAY_OF_WEEK  LIGHT_CONDITION  R